In [91]:
#載入套件
import pandas as pd
import numpy as np
import sklearn.svm
import random
import time
from sklearn import svm
from sklearn.model_selection import train_test_split



In [92]:
file_path='C:/Users/user/OneDrive/桌面/data .csv'



#data資料
#用pandas讀取資料
def data_prepare_1(file_path):
    df_file=pd.read_csv(file_path)
    Y=df_file['Diagnosis'].map({'M':-1,'B':1}).to_numpy()
    X=df_file.iloc[:,2:].to_numpy()
    return X, Y
X,Y=data_prepare_1(file_path)
















Please implement the Linear Classifier from scratch with the update rule in the slide. It means you cannot adopt any existing package like sklearn in this assignment. 

In [93]:
class LinearClassifier:
    def __init__(self) -> None:
        self.accl = 0
        self.acc2 = 0




    def linearClassifier(self,X,Y):
        convergenceFlag = False
        convergenceNumbers = 0
        iterations = 0
        #learning rate
        r = 0.001
        #array of number for shuffling
        order = np.arange(0,len(X),1)
        random.seed(1)
        W = np.random.rand(X.shape[1])
        b = np.random.random()
        start_time = time.time()

        while not convergenceFlag:
            iterations += 1

            random.shuffle(order)
            for i in order:
                F = X[i,:].reshape(len(X[i,:]))
                label = Y[i]

                #check if it is correct based on the current model
                prediction = 1 if (np.dot(W,F)+b) > 0 else 0
                if label*prediction < 0:
                    #update all the weights
                    for j in range (len(W)):
                        W[j] = W[j] + r*(F[j]*label)
                    b = b+ label
                else:
                    convergenceNumbers +=1
            if iterations > 1000 or convergenceNumbers == len(Y):
                convergenceFlag = True
        print('---{} seconds ---'.format(time.time() - start_time)) 
        #calculate the accuracy
        s = 0
        acc1=0
        for x,y in zip(X,Y):
            pred = 1 if (np.dot(W,x)+b) > 0 else -1
            if y==pred:
                s +=1
            acc1 = s / len(Y)
        print('--- accuracy: {} ---'.format(acc1)) 








In [94]:
LinearClassifier().linearClassifier(X,Y)

---1.8462395668029785 seconds ---
--- accuracy: 0.37258347978910367 ---


the "J=WX+b" Whencould be represented as the matrix form for the linear classifier, please find the solution by solving this equation using least-squared manner. Also, please implement it and make a comparison between this method and the previous one implemented in 2.

In [95]:
def LinearClassifierMatrixForm(X,Y):
    Y = Y.reshape((len(Y),1))
    w = np.zeros((X.shape[1], 1))
    b = np.random.random(size=(len(X),1))

    
    start_time = time.time()
    beta_hat = np.vstack((b,w))
    designMatrix = np.hstack((np.ones((len(X),1)),X))
    beta_hat =np.linalg.inv(designMatrix.T @ designMatrix) @ designMatrix.T @ Y

    print('---{} seconds ---'.format(time.time() - start_time)) 
    
    s=0
    preds = [1 if item >0 else -1 for item in (np.dot(designMatrix,beta_hat))]           
    for y,pred in zip(Y,preds):

        if y == pred:
            s +=1
        acc1 = s / len(Y)
    print('--- accuracy: {} ---'.format(acc1)) 

            



    
     



In [72]:
LinearClassifierMatrixForm(X,Y)

---0.0009984970092773438 seconds ---
--- accuracy: 0.9648506151142355 ---


Please implement the voted perception and make the performance comparison between methods.

In [73]:
class VotedPerceptron:
    def __init__(self, n_iter)-> None:
        self.n_iter = n_iter
        self.V = []  # store the weights
        self.C = []  # store the number of examples that set of weights got correct
        self.k = 0
    
    def fit(self, x, y):
        #change labels,0 ->-1,1-> 1
        for i in range(len(y)):
            if y[i] == 0:
                y[i] = -1
        k = 0
        v = [np.ones_like(x[0])]
        c = [0]
        #train n_iter epoches
        for epoch in range(self.n_iter): 
            #check if each X_i is classified right  
            for i in range(len(x)):
                pred = 1 if np.dot(v[k], x[i]) > 0 else -1 # checks the sing of v*k
                if pred == y[i]: # checks if the prediction matches the real Y
                    c[k] += 1 # increments c
                else:
                    v.append(np.add(v[k], np.dot(y[i], x[i])))
                    c.append(1)
                    k += 1
        # #change labels,0 ->-1,1-> 1
        for i in range(len(y)):
            if y[i] == -1:
                y[i] = 0          
        self.V = v
        self.C = c
        self.k = k

    #classifying
    def predict(self, X):
        preds = []
        for x in X:
            s = 0
            for w,c in zip(self.V,self.C):
                s = s + c*np.sign(np.dot(w,x))
            preds.append(np.sign(1 if s>= 0 else 0))
        return preds

In [74]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
v_perc = VotedPerceptron(n_iter=100)
v_perc.fit(X, Y)
Ypred = v_perc.predict(X)
Y_tolist = Y.tolist()
print('Accuracy score: ', accuracy_score(Y_tolist, Ypred))
print('Metrics Report')
print(classification_report(Y_tolist, Ypred))

Accuracy score:  0.9226713532513181
Metrics Report
              precision    recall  f1-score   support

           0       0.93      0.85      0.89       212
           1       0.92      0.96      0.94       357

    accuracy                           0.92       569
   macro avg       0.93      0.91      0.92       569
weighted avg       0.92      0.92      0.92       569



 With minimizing the ||w||^2, it should drive the marginal to be maximized as well. Please implement the linear classifier with the minimum ||w||^2 property and verify whether the margin of this version is larger than that of the conventional linear classifier or not. 

In [75]:
def SVM1( X, Y):
    Y = Y.reshape(len(Y),1)
    w = np.ones_like(X[0])
    b = np.random.random()
    #learning rate
    r = 0.01
    iter = 0
    start_time = time.time()
    while iter < 100:
        for i in range (len(X)):
            pred = 1 if (np.dot (w,X[i]) + b) > 0 else -1
            loss = np.linalg.norm(w,ord=2)**2
            delta = 1e-4
            if pred*Y[i] < 1:
                for j in range (len(w)):
                    w[j] -= r*(-Y[i]*X[i,j])*delta
                b = b + Y[i]
        iter += 1
        
    print('---{} seconds ---'.format(time.time() - start_time)) 

    #calculate the accuracy
    s = 0
    for x,y in zip(X,Y):
        pred = 1 if (np.dot(w,x)+b) > 0 else 0
        if y==pred:
           s +=1
    acc1 = s / len(Y)
    print('--- accuracy: {} ---'.format(acc1)) 

              




Based on 5, please add the slack variable term in the linear classifier and find the most effective weighting value C.

In [96]:
def SVM2( X, Y):
    Y = Y.reshape(len(Y),1)
    w = np.ones_like(X[0])
    b = 0
    #learning rate
    r = 0.01
    # constant of slack variable
    k=0.1
    #slack variable
    slackVariables = np.zeros(len(X))

    iter = 0
    start_time = time.time()
    while iter < 100:
        for i in range (len(X)):
            pred = 1 if (np.dot (w,X[i]) + b) >=0 else -1

            for k in range (len(X)):
                pred = 1 if (np.dot (w,X[k]) + b) >=0 else -1
                slackVariables[i]==max(0,1-Y[i]*pred)




            loss = np.linalg.norm(w,ord=2) + k*np.sum(slackVariables)
            delta= 1e-4

            if pred*Y[i] < 1:
                for j in range (len(w)):
                    w[j] -= r*(-Y[i]*X[i,j])*delta
                b = b + Y[i]
                
        iter += 1
        
    print('---{} seconds ---'.format(time.time() - start_time)) 

    #calculate the accuracy
    s = 0
    for x,y in zip(X,Y):
        pred = 1 if (np.dot(w,x)+b) > 0 else 0
        if y==pred:
           s +=1
    acc2 = s / len(Y)
    print('--- accuracy: {} ---'.format(acc2)) 
    

              




In [97]:

SVM1(X,Y)
SVM2(X,Y)


---2.59289813041687 seconds ---
--- accuracy: 0.5905096660808435 ---
---326.135071516037 seconds ---
--- accuracy: 0.012302284710017574 ---


Use any existing SVM package (libSVM, sklearn, or Matlab SVM), please make a performance comparison between the built-in SVM and your implementation.

In [77]:
X_train, X_test, y_train, y_test = train_test_split(X, Y,test_size=0.2,random_state=0)
clf=svm.SVC(kernel='rbf',C=1,gamma='auto')
clf.fit(X_train,y_train)

clf.predict(X_test)

print(clf.score(X_train,y_train))
print(clf.score(X_test, y_test))





1.0
0.5877192982456141


In [98]:
file_path='C:/Users/user/OneDrive/桌面/crx .csv'


#crx資料
def data_prepare_2(file_path):
    df_file = pd.read_csv(file_path)
    Y=df_file['label'].map({'+':1,'-':-1}).to_numpy()
    X=df_file.iloc[:,:15]
    X['att1']=X['att1'].map({'?':0,'a':-1,'b':1})
    X['att2']=X['att2']=X['att2'].replace('?',0)
    X['att2']=X['att2'].astype(float)
    X['att4']=X['att4'].map({'?':0,'l':0,'u':1,'y':2})
    X['att5']=X['att5'].map({'?':0,'g':1,'gg':2,'p':3})
    X['att6']=X['att6'].map({'?':0,'aa':-7,'c':-6,'cc':-5,'d':-4,'e':-3,'ff':-2,'i':-1,'j':1,'k':2,'m':3,'q':4,'r':5,'w':6,'x':7})
    X['att7']=X['att7'].map({'?':0,'bb':1,'dd':2,'ff':3,'h':4,'j':5,'n':6,'o':7,'v':8,'z':9})
    X['att9']=X['att9'].map({'t':-1,'f':1})
    X['att10']=X['att10'].map({'t':-1,'f':1})
    X['att12']=X['att12'].map({'t':-1,'f':1})
    X['att13']=X['att13'].map({'g':-1,'p':0,'s':1})
    X['att14']=X['att14']=X['att14'].replace('?',0)
    X['att14']=X['att14']=X['att14'].astype(float)
    X = X.to_numpy()
    return X,Y

X,Y=data_prepare_2(file_path)

Please implement the Linear Classifier from scratch with the update rule in the slide. It means you cannot adopt any existing package like sklearn in this assignment. 

In [99]:
class LinearClassifier:
    def __init__(self) -> None:
        self.accl = 0
        self.acc2 = 0




    def linearClassifier(self,X,Y):
        convergenceFlag = False
        convergenceNumbers = 0
        iterations = 0
        #learning rate
        r = 0.001
        #array of number for shuffling
        order = np.arange(0,len(X),1)
        random.seed(1)
        W = np.random.rand(X.shape[1])
        b = np.random.random()
        start_time = time.time()

        while not convergenceFlag:
            iterations += 1

            random.shuffle(order)
            for i in order:
                F = X[i,:].reshape(len(X[i,:]))
                label = Y[i]

                #check if it is correct based on the current model
                prediction = 1 if (np.dot(W,F)+b) > 0 else 0
                if label*prediction < 0:
                    #update all the weights
                    for j in range (len(W)):
                        W[j] = W[j] + r*(F[j]*label)
                    b = b+ label
                else:
                    convergenceNumbers +=1
            if iterations > 1000 or convergenceNumbers == len(Y):
                convergenceFlag = True
        print('---{} seconds ---'.format(time.time() - start_time)) 
        #calculate the accuracy
        s = 0
        acc1=0
        for x,y in zip(X,Y):
            pred = 1 if (np.dot(W,x)+b) > 0 else -1
            if y==pred:
                s +=1
            acc1 = s / len(Y)
        print('--- accuracy: {} ---'.format(acc1)) 








In [82]:
LinearClassifier().linearClassifier(X,Y)

---2.5536739826202393 seconds ---
--- accuracy: 0.5579710144927537 ---


When the "J=WX+b" could be represented as the matrix form for the linear classifier, please find the solution by solving this equation using least-squared manner. Also, please implement it and make a comparison between this method and the previous one implemented in 2.

In [83]:
def LinearClassifierMatrixForm(X,Y):
    Y = Y.reshape((len(Y),1))
    w = np.zeros((X.shape[1], 1))
    b = np.random.random(size=(len(X),1))

    
    start_time = time.time()
    beta_hat = np.vstack((b,w))
    designMatrix = np.hstack((np.ones((len(X),1)),X))
    beta_hat =np.linalg.inv(designMatrix.T @ designMatrix) @ designMatrix.T @ Y

    print('---{} seconds ---'.format(time.time() - start_time)) 
    
    s=0
    preds = [1 if item >0 else -1 for item in (np.dot(designMatrix,beta_hat))]           
    for y,pred in zip(Y,preds):

        if y == pred:
            s +=1
        acc1 = s / len(Y)
    print('--- accuracy: {} ---'.format(acc1))            


In [84]:
LinearClassifierMatrixForm(X,Y)

---0.12566590309143066 seconds ---
--- accuracy: 0.8594202898550725 ---


Please implement the voted perception and make the performance comparison between methods.

In [85]:
class VotedPerceptron:
    def __init__(self, n_iter)-> None:
        self.n_iter = n_iter
        self.V = []  # store the weights
        self.C = []  # store the number of examples that set of weights got correct
        self.k = 0
    
    def fit(self, x, y):
        #change labels,0 ->-1,1-> 1
        for i in range(len(y)):
            if y[i] == 0:
                y[i] = -1
        k = 0
        v = [np.ones_like(x[0])]
        c = [0]
        #train n_iter epoches
        for epoch in range(self.n_iter): 
            #check if each X_i is classified right  
            for i in range(len(x)):
                pred = 1 if np.dot(v[k], x[i]) > 0 else -1 # checks the sing of v*k
                if pred == y[i]: # checks if the prediction matches the real Y
                    c[k] += 1 # increments c
                else:
                    v.append(np.add(v[k], np.dot(y[i], x[i])))
                    c.append(1)
                    k += 1
        # #change labels,0 ->-1,1-> 1
        for i in range(len(y)):
            if y[i] == -1:
                y[i] = 0          
        self.V = v
        self.C = c
        self.k = k

    #classifying
    def predict(self, X):
        preds = []
        for x in X:
            s = 0
            for w,c in zip(self.V,self.C):
                s = s + c*np.sign(np.dot(w,x))
            preds.append(np.sign(1 if s>= 0 else 0))
        return preds

In [86]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
v_perc = VotedPerceptron(n_iter=100)
v_perc.fit(X, Y)
Ypred = v_perc.predict(X)
Y_tolist = Y.tolist()
print('Accuracy score: ', accuracy_score(Y_tolist, Ypred))
print('Metrics Report')
print(classification_report(Y_tolist, Ypred))

Accuracy score:  0.6434782608695652
Metrics Report
              precision    recall  f1-score   support

           0       0.68      0.66      0.67       383
           1       0.60      0.62      0.61       307

    accuracy                           0.64       690
   macro avg       0.64      0.64      0.64       690
weighted avg       0.65      0.64      0.64       690



With minimizing the ||w||^2, it should drive the marginal to be maximized as well. Please implement the linear classifier with the minimum ||w||^2 property and verify whether the margin of this version is larger than that of the conventional linear classifier or not.

In [100]:
def SVM1( X, Y):
    Y = Y.reshape(len(Y),1)
    w = np.ones_like(X[0])
    b = np.random.random()
    #learning rate
    r = 0.01
    iter = 0
    start_time = time.time()
    while iter < 100:
        for i in range (len(X)):
            pred = 1 if (np.dot (w,X[i]) + b) > 0 else -1
            loss = np.linalg.norm(w,ord=2)**2
            delta = 1e-4
            if pred*Y[i] < 1:
                for j in range (len(w)):
                    w[j] -= r*(-Y[i]*X[i,j])*delta
                b = b + Y[i]
        iter += 1
        
    print('---{} seconds ---'.format(time.time() - start_time)) 

    #calculate the accuracy
    s = 0
    for x,y in zip(X,Y):
        pred = 1 if (np.dot(w,x)+b) > 0 else 0
        if y==pred:
           s +=1
    acc1 = s / len(Y)
    print('--- accuracy: {} ---'.format(acc1)) 

Based on 5, please add the slack variable term in the linear classifier and find the most effective weighting value C.

In [101]:
def SVM2( X, Y):
    Y = Y.reshape(len(Y),1)
    w = np.ones_like(X[0])
    b = 0
    #learning rate
    r = 0.01
    # constant of slack variable
    k=0.1
    #slack variable
    slackVariables = np.zeros(len(X))

    iter = 0
    start_time = time.time()
    while iter < 100:
        for i in range (len(X)):
            pred = 1 if (np.dot (w,X[i]) + b) >=0 else -1

            for k in range (len(X)):
                pred = 1 if (np.dot (w,X[k]) + b) >=0 else -1
                slackVariables[i]==max(0,1-Y[i]*pred)




            loss = np.linalg.norm(w,ord=2) + k*np.sum(slackVariables)
            delta= 1e-4

            if pred*Y[i] < 1:
                for j in range (len(w)):
                    w[j] -= r*(-Y[i]*X[i,j])*delta
                b = b + Y[i]
                
        iter += 1
        
    print('---{} seconds ---'.format(time.time() - start_time)) 

    #calculate the accuracy
    s = 0
    for x,y in zip(X,Y):
        pred = 1 if (np.dot(w,x)+b) > 0 else 0
        if y==pred:
           s +=1
    acc2 = s / len(Y)
    print('--- accuracy: {} ---'.format(acc2)) 

In [102]:

SVM1(X,Y)
SVM2(X,Y)


---2.9647107124328613 seconds ---
--- accuracy: 0.20434782608695654 ---
---487.6715142726898 seconds ---
--- accuracy: 0.42028985507246375 ---


Use any existing SVM package (libSVM, sklearn, or Matlab SVM), please make a performance comparison between the built-in SVM and your implementation.

In [90]:
X_train, X_test, y_train, y_test = train_test_split(X, Y,test_size=0.2,random_state=0)
clf=svm.SVC(kernel='rbf',C=1,gamma='auto')
clf.fit(X_train,y_train)

clf.predict(X_test)

print(clf.score(X_train,y_train))
print(clf.score(X_test, y_test))



1.0
0.5507246376811594
